# Notebook 02 - Vannmiljo Conversion

Sam Welch  
December 16, 2025

# Setup

# To Do:

-   [ ] Expand compartments from just aquatic/freshwater
-   [ ] Investigate some very strange detection and quantification limits
-   [ ] Add probable contamination source
-   [ ] More sanity checking of extracted values
-   [ ] Move boring code to targets
-   [ ] Fix export **again**.

## Libraries

In [ ]:

library(dplyr)



Attaching package: 'dplyr'

The following objects are masked from 'package:stats':

    filter, lag

The following objects are masked from 'package:base':

    intersect, setdiff, setequal, union


Attaching package: 'lubridate'

The following objects are masked from 'package:base':

    date, intersect, setdiff, union

Linking to GEOS 3.13.1, GDAL 3.11.0, PROJ 9.6.0; sf_use_s2() is TRUE


Attaching package: 'eDataDRF'

The following objects are masked from 'package:STOPeData':

    abbreviate_string, altitude_units_vocabulary,
    analytical_protocols_vocabulary, areas_vocabulary,
    coordinate_systems_vocabulary, countries_vocabulary,
    dummy_parameters_vocabulary, environ_compartments_sub_vocabulary,
    environ_compartments_vocabulary, extraction_protocols_vocabulary,
    fractionation_protocols_vocabulary, gender_vocabulary,
    generate_protocol_id, geographic_features_sub_vocabulary,
    geographic_features_vocabulary, get_dataset_display_name,
    initialise_biota_tibble, initialise_campaign_tibble,
    initialise_compartments_tibble, initialise_CREED_data_tibble,
    initialise_CREED_scores_tibble, initialise_measurements_tibble,
    initialise_methods_tibble, initialise_parameters_tibble,
    initialise_references_tibble, initialise_samples_tibble,
    initialise_sites_tibble, lifestage_vocabulary,
    measured_categories_vocabulary, measured_flags_vocabulary,
    measured_types_vocabular

here() starts at C:/Users/SAW/Local Documents/EXPECT AEP/EXPECT_AEP_R_Project

here() starts at C:/Users/SAW/Local Documents/EXPECT AEP/EXPECT_AEP_R_Project

## Configuration

See @tbl-config-filters and @tbl-config-metadata for a summary of the parameters of this extraction from Vannmiljø.

In [ ]:

config <- list(
  # File paths
  files = list(
    vm_copper = here("data/raw/vannmiljo/Vm_Copper_2025.12.05.xlsx"),
    vm_sites_1 = here("data/raw/vannmiljo/Vm_Copper_Sites_2025.12.05-1.xlsx"),
    vm_sites_2 = here("data/raw/vannmiljo/Vm_Copper_Sites_2025.12.05-2.xlsx"),
    vm_sites_3 = here("data/raw/vannmiljo/Vm_Copper_Sites_2025.12.05-3.xlsx"),
    lookup_medium = here("data/clean/Vm_medium_lookup_matrix_filled.csv"),
    lookup_vannkategori = here("data/clean/vm_sites_codes_lookup.csv"),
    lookup_methods = here("data/clean/vm_methods_lookup_filled.csv"),
    methods_analysis = here(
      "data/raw/vannmiljo/Vannmiljø_Analysemetode_2025-12-15.xlsx"
    ),
    methods_sampling = here(
      "data/raw/vannmiljo/Vannmiljø_Prøvetakingsmetode_2025-12-15.xlsx"
    )
  ),

  # Filter criteria
  filters = list(
    date_start = as.Date("2010-01-01"),
    date_end = as.Date("2025-12-05"),
    exclude_sites = "Svalbard, ENSB-Kilde 2"
  ),

  # Campaign metadata
  metadata = list(
    campaign_name = "Vannmiljø Copper Monitoring 2010-2025",
    campaign_short = "Vm_2010_2025",
    reference_id = "VannmiljøCopper2010-2025",
    entered_by = "Sam Welch",
    organisation = "Miljødirektoratet"
  )
)


In [ ]:

config$filters |> as_tibble() |> knitr::kable()


  date_start   date_end     exclude_sites
  ------------ ------------ ------------------------
  2010-01-01   2025-12-05   Svalbard, ENSB-Kilde 2


In [ ]:

config$metadata |> as_tibble() |> knitr::kable()


  ------------------------------------------------------------------------------------------------------
  campaign_name             campaign_short   reference_id               entered_by   organisation
  ------------------------- ---------------- -------------------------- ------------ -------------------
  Vannmiljø Copper          Vm_2010_2025     VannmiljøCopper2010-2025   Sam Welch    Miljødirektoratet
  Monitoring 2010-2025                                                               

  ------------------------------------------------------------------------------------------------------


## Helper Functions

In [ ]:
# Convert Vannmiljø operator codes to eData flags
vm_convert_operator <- function(col) {
  case_match(
    col,
    "=" ~ "",
    "<" ~ "< LOQ",
    ">" ~ "What is going on here. Why higher than?",
    "ND" ~ "< LOD"
  )
}


# Load Raw Data

## Measurements Data

Downloaded 2025.12.05, data for 2025.01.01 - 2025.12.05, all kommune, all media, all campaigns Search for Kobber & Kobberpyrition: 138615 hits NOTE: Data may extend beyond 2025 date range despite request parameters

In [ ]:

# Downloaded 2025.12.05, data for 2025.01.01 - 2025.12.05, all kommune, all media, all campaigns
# Search for Kobber & Kobberpyrition: 138615 hits
# NOTE: Data may extend beyond 2025 date range despite request parameters

if (!exists("vm_raw_copper")) {
  vm_raw_copper <- read_excel(
    path = config$files$vm_copper,
    sheet = 1,
    guess_max = 138615 # Avoid automatic type conversion warnings
  )
  message(glue("Loaded {nrow(vm_raw_copper)} copper measurements"))
}


Loaded 138615 copper measurements

Generated by summarytools 1.1.4 ( R version 4.5.2) 2025-12-19

## Sites Data

NOTE: Vannmiljø export limit requires splitting site codes into multiple files Procedure: 1. Extract unique site codes: vm_raw_copper\$Vannlok_kode \|\> unique() \|\> writeClipboard() 2. Replace newlines with commas: Ctrl+H find “” replace with “,” 3. Split into chunks (Vannmiljø fails silently above unknown threshold) 4. Download as separate Excel files

In [ ]:

vm_raw_sites <- read_excel(
  config$files$vm_sites_1,
  guess_max = 10000
) |>
  add_row(read_excel(
    config$files$vm_sites_2,
    guess_max = 10000
  )) |>
  add_row(read_excel(
    config$files$vm_sites_3,
    guess_max = 10000
  ))

message(glue("Loaded {nrow(vm_raw_sites)} site records"))


Loaded 30541 site records

## Lookup Tables

This part is very messy - we generated a bunch of lookup tables (semi-reproducibly) and filled them by hand/with Claude. I don’t have a lot of confidence in any of them.

In [ ]:

# Medium ID lookup
# Maps Vannmiljø Medium_id to eData compartment/species/tissue fields
vm_lookup_medium_raw <- read_csv(config$files$lookup_medium)


Rows: 54 Columns: 12
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
chr (10): MediumID, Name, Description, ENVIRON_COMPARTMENT, ENVIRON_COMPARTM...
dbl  (1): n
lgl  (1): SAMPLE_SPECIES

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.

e.g.:
  dat <- vroom(...)
  problems(dat)

Rows: 9 Columns: 8
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
chr (7): VannkategoriID, Name, Description, ENVIRON_COMPARTMENT, ENVIRON_COM...
dbl (1): n

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.

Rows: 14 Columns: 5
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
chr (3): PROTOCOL_CATEGORY, PROTOCOL_NAME, PROTOCOL_COMMENT
dbl (1): PROTOCOL_ID
lgl (1): CAMPAIGN_NAME

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.

Loaded all lookup tables

# Data Processing

## Merge Datasets

We join sites and measurements by `"Vannlok_kode" = "Vannlokalitetskode"`, then our compartment/etc. lookups.

In [ ]:

# Join sites to measurements
vm_merged <- vm_raw_copper |>
  left_join(
    vm_raw_sites,
    by = c("Vannlok_kode" = "Vannlokalitetskode")
  )

message(glue("Merged sites: {nrow(vm_merged)} rows"))


Merged sites: 138615 rows

Added lookup tables

## Apply Filters

As above, we filter data very heavily here, so we don’t have to spend a lot of time on edge cases.

Data Quality Filters Applied: 2025-12-16

Included: - Freshwater only (ENVIRON_COMPARTMENT_SUB = “Freshwater”) - Point sites only (no polygons) - No biota (LatinskNavn_id == 0) - Mainland Norway only (excluded Svalbard) - Dates: (see config)

Excluded (~43% of data): - Marine/brackish water - Sediment samples - Biota samples - Svalbard sites (n=1)

In [ ]:

vm_filtered <- vm_merged |>
  # Add sampling date as Date type
  mutate(SAMPLING_DATE = as.Date(Tid_provetak)) |>
  # Apply compartment filters
  filter(
    ENVIRON_COMPARTMENT_vkat == "Aquatic",
    ENVIRON_COMPARTMENT_medium == "Aquatic",
    ENVIRON_COMPARTMENT_SUB_vkat == "Freshwater",
    ENVIRON_COMPARTMENT_SUB_medium == "Freshwater"
  ) |>
  # Apply site type filters
  filter(
    Objekttype == "point",
    LatinskNavn_id == 0,
    Vannlokalitetsnavn != config$filters$exclude_sites
  ) |>
  # Apply date filter
  filter(SAMPLING_DATE >= config$filters$date_start)

message(glue(
  "After filtering: {nrow(vm_filtered)} rows ({round(100*nrow(vm_filtered)/nrow(vm_merged), 1)}% of merged data)"
))


After filtering: 48384 rows (34.9% of merged data)

## Quality Checks

We check for conflicts between our two lookups here - because we look up compartments, sites, biota, etc. based on both site `Vannlokalkode` and measurments’ `MediumID`, it’s possible that we might end up with conflicts and/or cases where neither of these give us the relevant data and we have to guess from context. Right now we filter down to only freshwater cases, hence the fact that we don’t get hits below.

In [ ]:

# Check for conflicts between lookup sources
conflict_check <- vm_filtered |>
  group_by(
    ENVIRON_COMPARTMENT_vkat,
    ENVIRON_COMPARTMENT_medium,
    ENVIRON_COMPARTMENT_SUB_vkat,
    ENVIRON_COMPARTMENT_SUB_medium
  ) |>
  reframe(count = n()) |>
  mutate(
    compartment_conflict = ENVIRON_COMPARTMENT_vkat !=
      ENVIRON_COMPARTMENT_medium,
    subcompartment_conflict = ENVIRON_COMPARTMENT_SUB_vkat !=
      ENVIRON_COMPARTMENT_SUB_medium
  )

if (
  any(conflict_check$compartment_conflict) ||
    any(conflict_check$subcompartment_conflict)
) {
  warning("Conflicts detected between vannkategori and medium lookups")
  print(
    conflict_check |> filter(compartment_conflict | subcompartment_conflict)
  )
}

# Check geographic feature consistency
geo_check <- vm_filtered |>
  group_by(
    SITE_GEOGRAPHIC_FEATURE_vkat,
    SITE_GEOGRAPHIC_FEATURE_medium,
    SITE_GEOGRAPHIC_FEATURE_SUB_vkat,
    SITE_GEOGRAPHIC_FEATURE_SUB_medium
  ) |>
  reframe(count = n()) |>
  mutate(
    geo_conflict = SITE_GEOGRAPHIC_FEATURE_vkat !=
      SITE_GEOGRAPHIC_FEATURE_medium,
    geo_sub_conflict = SITE_GEOGRAPHIC_FEATURE_SUB_vkat !=
      SITE_GEOGRAPHIC_FEATURE_SUB_medium
  )

message(glue(
  "Geographic feature conflicts: {sum(geo_check$geo_conflict | geo_check$geo_sub_conflict, na.rm = TRUE)}"
))


Geographic feature conflicts: 0

# Create eData Tables

## Campaign

In [ ]:

edata_campaign <- initialise_campaign_tibble() |>
  add_row(
    CAMPAIGN_NAME_SHORT = config$metadata$campaign_short,
    CAMPAIGN_NAME = config$metadata$campaign_name,
    CAMPAIGN_START_DATE = config$filters$date_start,
    CAMPAIGN_END_DATE = config$filters$date_end,
    RELIABILITY_SCORE = NA_character_,
    RELIABILITY_EVAL_SYS = NA_character_,
    CONFIDENTIALITY_EXPIRY_DATE = as.Date(NA),
    ORGANISATION = config$metadata$organisation,
    ENTERED_BY = config$metadata$entered_by,
    ENTERED_DATE = as.Date(Sys.Date()),
    CAMPAIGN_COMMENT = "Copper and copper pyrithione measurements from Vannmiljø database covering all Norwegian municipalities and media types"
  )

message("Created campaign table")


Created campaign table

## Reference

In [ ]:

edata_reference <- initialise_references_tibble() |>
  add_row(
    REFERENCE_ID = config$metadata$reference_id,
    REFERENCE_TYPE = "Database",
    DATA_SOURCE = "Vannmiljø",
    AUTHOR = config$metadata$organisation,
    TITLE = "Vannmiljø Database - Copper and Copper Pyrithione Data",
    YEAR = 2025L,
    ACCESS_DATE = config$filters$date_end,
    PERIODICAL_JOURNAL = NA_character_,
    VOLUME = NA_integer_,
    ISSUE = NA_integer_,
    PUBLISHER = NA_character_,
    INSTITUTION = config$metadata$organisation,
    DOI = NA_character_,
    URL = "https://www.vannmiljo.no/",
    ISBN_ISSN = NA_character_,
    EDITION = NA_character_,
    DOCUMENT_NUMBER = NA_character_,
    REF_COMMENT = glue(
      "Downloaded {config$filters$date_end}, data for {config$filters$date_start} to {config$filters$date_end}, ",
      "all kommune, all media, all campaigns. Search for Kobber & Kobberpyrition. 138615 hits."
    )
  )

message("Created reference table")


Created reference table

## Parameters

In [ ]:

# No copper pyrithione reported for 2025, only copper
edata_parameters <- initialise_parameters_tibble() |>
  add_row(
    PARAMETER_TYPE = NA_character_,
    PARAMETER_TYPE_SUB = NA_character_,
    MEASURED_TYPE = NA_character_,
    PARAMETER_NAME = "Copper",
    PARAMETER_NAME_SUB = NA_character_,
    INCHIKEY_SD = NA_character_,
    PUBCHEM_CID = NA_integer_,
    CAS_RN = NA_character_,
    ENTERED_BY = glue("{config$metadata$entered_by} from Vannmiljø"),
    PARAMETER_COMMENT = NA_character_
  )

message("Created parameters table")


Created parameters table

## Sites

In [ ]:

# Extract unique sites with relevant metadata
vm_sites_unique <- vm_filtered |>
  select(
    Vannlok_kode,
    Vannlokalitetsnavn,
    Beskrivelse,
    `UTM33 Ost (X)`,
    `UTM33 Nord (Y)`,
    `Knytt til påvirkning`,
    SITE_GEOGRAPHIC_FEATURE_medium,
    SITE_GEOGRAPHIC_FEATURE_vkat,
    SITE_GEOGRAPHIC_FEATURE_SUB_medium,
    SITE_GEOGRAPHIC_FEATURE_SUB_vkat
  ) |>
  distinct() |>
  # Clean up emission source
  mutate(
    Emission_Source = case_match(
      `Knytt til påvirkning`,
      c("Industri", "INDUSTRI") ~ "Industrial",
      c("Akvakultur", "AKVAKULTUR") ~ "Aquaculture",
      .default = NA_character_
    )
  )

message(glue("Extracted {nrow(vm_sites_unique)} unique sites"))


Extracted 4984 unique sites

Warning in geom_map(data = world_map, map = world_map, aes(long, lat, map_id =
region)): Ignoring unknown aesthetics: x and y

# A tibble: 4,984 × 14
   SITE_CODE SITE_NAME SITE_GEOGRAPHIC_FEAT…¹ SITE_GEOGRAPHIC_FEAT…² COUNTRY_ISO
   <glue>    <glue>    <chr>                  <chr>                  <chr>      
 1 Vannmilj… Vannmilj… Lake, pond, pool, res… Water column, pelagic… Norway     
 2 Vannmilj… Vannmilj… Lake, pond, pool, res… Water column, pelagic… Norway     
 3 Vannmilj… Vannmilj… Lake, pond, pool, res… Water column, pelagic… Norway     
 4 Vannmilj… Vannmilj… Lake, pond, pool, res… Water column, pelagic… Norway     
 5 Vannmilj… Vannmilj… Lake, pond, pool, res… Water column, pelagic… Norway     
 6 Vannmilj… Vannmilj… Lake, pond, pool, res… Water column, pelagic… Norway     
 7 Vannmilj… Vannmilj… River, stream, canal   Water column, pelagic… Norway     
 8 Vannmilj… Vannmilj… River, stream, canal   Water column, pelagic… Norway     
 9 Vannmilj… Vannmilj… River, stream, canal   Water column, pelagic… Norway     
10 Vannmilj… Vannmilj… River, stream, canal   Water column, pelagic… Norway     
# ℹ 4

Created sites table: 4984 sites

## Methods

In [ ]:

# Methods Mapping
# NOTE: Methods mapping simplified for initial conversion
# Current approach:
# - Simple 1:1 mapping from Vannmiljø codes
# - Filtered = assumed 0.45 µm
# - Total concentration only (no fractionation detail)
#
# Known limitations:
# - "UKJENT" (Unknown) methods not properly handled
# - Fractionation protocol oversimplified
# - Extraction protocols not captured
# - Some method details may be in Norwegian standards not reviewed

# Filter methods to those actually used in freshwater data
vm_methods_used_analysis <- vm_lookup_analysis |>
  filter(AnalysisMethodID %in% vm_filtered$Analysemetode_id)

vm_methods_used_sampling <- vm_lookup_sampling |>
  filter(
    SamplingMethodID %in% vm_filtered$Provetakmetode_id,
    str_detect(Name, "erskvann"), # Freshwater only
    !str_detect(Name, "sediment") # Exclude sediment
  )

# Merge and categorize
edata_methods <- vm_methods_used_sampling #
# TODO: Renable methods.
# bind_rows(vm_methods_used_analysis) |>
# mutate(
#   ID = case_when(
#     !is.na(AnalysisMethodID) ~ AnalysisMethodID,
#     !is.na(SamplingMethodID) ~ SamplingMethodID,
#     .default = NA_character_
#   ),
#   Category = case_when(
#     !is.na(AnalysisMethodID) ~ "Analysis",
#     !is.na(SamplingMethodID) ~ "Sampling",
#     .default = NA_character_
#   ),
#   .keep = "unused"
# ) |>
# # Join with eData mapping
# left_join(
#   vm_lookup_methods,
#   by = c("PROTOCOL_ID", "Category")
# ) |>
# group_by(PROTOCOL_CATEGORY) |>
# mutate(
#   CAMPAIGN_NAME = config$metadata$campaign_name,
#   PROTOCOL_ID = glue("{substr(PROTOCOL_CATEGORY, 1, 1)}{row_number()}-Vm2025")
# ) |>
# ungroup()

message(glue("Created methods table: {nrow(edata_methods)} protocols"))


Created methods table: 5 protocols

## Measurements

In [ ]:

# Create measurements table
# NOTE: All freshwater values reported as EnhetID = 8 (µg/L)

edata_measurements <- vm_filtered |>
  mutate(
    # Core identifiers
    SITE_CODE = glue("Vannmiljø_{Vannlok_kode}"),
    PARAMETER_NAME = "Copper",
    SAMPLING_DATE = as.character(SAMPLING_DATE),
    CAMPAIGN_NAME_SHORT = config$metadata$campaign_short,
    REFERENCE_ID = config$metadata$reference_id,

    # Compartment information
    ENVIRON_COMPARTMENT = ENVIRON_COMPARTMENT_medium,
    ENVIRON_COMPARTMENT_SUB = ENVIRON_COMPARTMENT_SUB_medium,

    # Parameter classification
    PARAMETER_TYPE = "Stressor",
    MEASURED_TYPE = "Concentration",

    # Sample information
    SUBSAMPLE = "1",
    SAMPLE_ID = "Fake ID", # FIXME: Needs proper sample ID generation

    # Measurement values
    MEASURED_FLAG = vm_convert_operator(Operator),
    MEASURED_VALUE = Verdi,
    MEASURED_UNIT = "µg/L",
    MEASURED_N = 1,

    # Uncertainty (not reported in Vannmiljø)
    UNCERTAINTY_TYPE = "Not reported",
    UNCERTAINTY_UPPER = NA_real_,
    UNCERTAINTY_LOWER = NA_real_,

    # Detection limits
    LOQ_VALUE = Kvantifiseringsgrense,
    LOQ_UNIT = "µg/L",
    LOD_VALUE = Deteksjonsgrense,
    LOD_UNIT = "µg/L",

    # Protocol references (FIXME: Need proper protocol ID mapping)
    SAMPLING_PROTOCOL = "1",
    EXTRACTION_PROTOCOL = "2",
    FRACTIONATION_PROTOCOL = "3",
    ANALYTICAL_PROTOCOL = "4",

    # Comments
    MEASUREMENT_COMMENT = Kommentar,

    .keep = "none"
  )

# Validate against eData schema
initialise_measurements_tibble() |> add_row(edata_measurements)


# A tibble: 48,384 × 27
   SITE_CODE       PARAMETER_NAME SAMPLING_DATE ENVIRON_COMPARTMENT_…¹ SUBSAMPLE
   <glue>          <chr>          <chr>         <chr>                  <chr>    
 1 Vannmiljø_053-… Copper         2017-08-09    Freshwater             1        
 2 Vannmiljø_053-… Copper         2017-09-12    Freshwater             1        
 3 Vannmiljø_053-… Copper         2017-04-26    Freshwater             1        
 4 Vannmiljø_053-… Copper         2017-05-18    Freshwater             1        
 5 Vannmiljø_053-… Copper         2017-06-14    Freshwater             1        
 6 Vannmiljø_053-… Copper         2017-07-11    Freshwater             1        
 7 Vannmiljø_053-… Copper         2018-04-16    Freshwater             1        
 8 Vannmiljø_053-… Copper         2018-05-22    Freshwater             1        
 9 Vannmiljø_053-… Copper         2018-08-07    Freshwater             1        
10 Vannmiljø_053-… Copper         2018-06-19    Freshwater             1        
# ℹ 

Created measurements table: 48384 measurements

# Export

## Create Export

In [ ]:

# Assemble datasets for export
export_datasets <- list(
  edata_sites = edata_sites,
  edata_campaign = edata_campaign,
  edata_reference = edata_reference,
  edata_parameters = edata_parameters,
  edata_measurements = edata_measurements
)

# Export as ZIP
# TODO: Disabled
# if (
#   !file.exists(here(
#     "data",
#     "raw",
#     "eData",
#     paste0(config$metadata$campaign_short, ".zip")
#   ))
# ) {
#   export_path <- export_campaign_zip(
#     dataset_list = export_datasets,
#     campaign_name = config$metadata$campaign_short,
#     output_path = here("data", "raw", "eData")
#   )
#   message(glue("Export complete: {export_path}"))
# } else {
#   message("Export file already exists, skipping")
# }

# message(glue("Export complete: {export_path}"))


## Species Data Coverage

None of these are very informative yet, because we don’t filter down to stuff that’s definitely biota first.

In [ ]:

# Species group coverage
species_group_coverage <- vm_merged |>
  group_by(SPECIES_GROUP_medium) |>
  reframe(count = n()) |>
  mutate(
    flag = case_when(
      SPECIES_GROUP_medium %in% c("Not reported", "", NA) ~ "Missing",
      .default = NA_character_
    )
  ) |>
  arrange(desc(count))

datatable(species_group_coverage, caption = "Species Group Coverage")
